In [1]:
!pip install --upgrade langchain
!pip install -q langchain-gigachat
!pip install -q pyTelegramBotAPI

In [19]:
from langchain.chains import ConversationChain
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
import telebot
from time import sleep
from langchain_gigachat import GigaChat

In [20]:
llm = GigaChat(
    # model="GigaChat-Max",
    credentials="LLM_API_Key",
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False,
)

bot = telebot.TeleBot("Bot_API_key")

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a friendly and knowledgeable restaurant assistant. "
     "Answer only questions about food, restaurants, menus, and dining tips. "
     "If the question is unrelated to dining, reply with: "
     "'Sorry, I can only help with food, restaurants, and dining information.'"
    ),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

user_conversations = {}

In [2]:
# Handling non-text messages
@bot.message_handler(content_types=['audio','video','document','photo','sticker','voice','location','contact'])
def not_text(message):
    bot.send_message(message.chat.id, "I only work with text messages!")

# Reset conversation history
@bot.message_handler(commands=['reset'])
def reset_memory(message):
    user_id = message.chat.id
    if user_id in user_conversations:
        user_conversations[user_id].memory.clear()
    bot.send_message(user_id, "Conversation history has been reset.")

# Handling text messages
@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    user_id = message.chat.id

    # Create ConversationChain for a new user
    if user_id not in user_conversations:
        user_conversations[user_id] = ConversationChain(
            llm=llm,
            verbose=True,
            memory=ConversationBufferMemory(
                return_messages=True,
                memory_key="history",
                k=10  # keep last 10 messages
            ),
            prompt=prompt
        )
    
    conversation = user_conversations[user_id]

    try:
        response = conversation.predict(input=message.text)
    except Exception:
        response = "Sorry, an error occurred. Please try again."
    
    bot.send_message(user_id, response)
    sleep(1)

# Start the bot
bot.polling(none_stop=True)